In [17]:
import sys
sys.path.append('../')
import src.prompts as prompts
import pandas as pd

In [2]:
prompts.authenticate_goose("../config.json")

In [3]:
def read_txt(filename: str) -> str:
    with open(filename) as f:
        return f.read()

In [4]:
import re
baseprompt = read_txt("question_prompt.txt")
baseprompt = re.sub(r'^.*?I', 'I', baseprompt)
prompt = baseprompt + "Q: What significant event happened in October 2006?\n"


In [5]:
import random

months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
years = ["2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"]
def create_date(month, year):
    return month + " " + year

def create_dates(years, n=3):
    dates = []
    for year in years:
        for _ in range(n):
            month = random.choice(months)
            dates.append(create_date(month, year))
    return set(dates)

def create_prompt(baseprompt: str, date: str):
    return baseprompt + f"Q: What significant event happened in {date}?\n"

all_dates = create_dates(years)
all_prompts = {date: create_prompt(baseprompt, date) for date in all_dates}

In [14]:
testytest = {key: prompts.generate_n_prompts(prompt, model_name="gpt-j-6b", n_completions=3, stop_token="\n") for key, prompt in all_prompts.items()}

In [24]:
def clean_answer(answer: str):
    """
    Removes the A: from the answer and splits by first newline 
    """
    answer = answer.replace("A: ", "")
    answer = answer.split("\n")[0]
    
    return answer

clean_answers = {key: [clean_answer(answer) for answer in answers] for key, answers in testytest.items()}

In [34]:
def answers_to_df(answers: dict) -> pd.DataFrame:
    answer_df = pd.DataFrame.from_dict(answers, orient='index').reset_index()
    melted_df = answer_df.melt(id_vars='index', value_name='answer') 

    return melted_df

answers_to_df(clean_answers)


,index,variable,answer
0,March 2024,0,There will be a world religious war.
1,April 2020,0,The 2020 United States presidential election o...
2,September 2023,0,The St. Louis Rams will move to Los Angeles fo...
3,November 2022,0,I am unable to tell you what happened.
4,October 2021,0,Born in Alabama
...,...,...,...
58,July 2018,2,"In July 2018, the Grammy Awards were held in L..."
59,June 2022,2,"According to the 100-year clock on the wall, t..."
60,October 2019,2,The rest of the world stopped questioning huma...
61,April 2018,2,The first test-firing of a Minuteman III inter...
